### Imports and Data Loading

In [17]:
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.models import Model
from keras.utils import load_img, img_to_array
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
import cv2
import os
from PIL import Image
import datetime


In [2]:
def load_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (64, 64))  # Consistent size
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def load_images(image_paths):
    images = []
    for path in tqdm(image_paths, desc="Loading images"):
        img = load_image(path)
        if img is not None:  # Check if image was loaded successfully
            images.append(img)
        else:
            print(f"Failed to load image: {path}")
    
    print("Images Loaded. Stacking into numpy array...")
    
    # Convert to numpy array with explicit shape
    return np.stack(images)  # Use np.stack with progress tracking

#### Load Images

In [3]:
# Image data loading strings
train_image_label_csv = '/Users/ericcui/repos/imagenette/imagenette2-320/train_imagenette.csv'
val_image_label_csv = "/Users/ericcui/repos/imagenette/imagenette2-320/val_imagenette.csv"
path_prefix = "/Users/ericcui/repos/imagenette/imagenette2-320/"

# Label to index mappings
label_index_mappings = {
    "n01440764": 0,
    "n02102040": 1,
    "n02979186": 2,
    "n03000684": 3,
    "n03028079": 4,
    "n03394916": 5,
    "n03417042": 6,
    "n03425413": 7,
    "n03445777": 8,
    "n03888257": 9,
}

# Label to class mappings
class_mappings = {
    "n01440764": "tench",
    "n02102040": "English springer",
    "n02979186": "cassette player",
    "n03000684": "chain saw",
    "n03028079": "church",
    "n03394916": "French horn",
    "n03417042": "garbage truck",
    "n03425413": "gas pump",
    "n03445777": "golf ball",
    "n03888257": "parachute"
}

In [ ]:
# Loading training dataframe
train_df = pd.read_csv(train_image_label_csv)

# Loading training images
image_paths = train_df['path'].tolist()
image_paths = [path_prefix + path for path in image_paths]
train_images = load_images(image_paths)


In [ ]:
# Loading validation dataframe
test_df = pd.read_csv(val_image_label_csv)

# Loading validatino images
test_image_paths = test_df['path'].tolist()
test_image_paths = [path_prefix + path for path in test_image_paths]
test_images = load_images(test_image_paths)

#### Load Training Labels

In [6]:
train_labels = train_df['noisy_labels_0'].tolist()
train_labels = np.array([label_index_mappings[label] for label in train_labels])

In [7]:
test_labels = test_df['noisy_labels_0'].tolist()
test_labels = np.array([label_index_mappings[label] for label in test_labels])

### Rendering Image and Label Sample

In [ ]:
class_names = ['tench', 'English springer', 'cassette player', 'chain saw', 'church', 'French horn', 'garbage truck', 'gas pump', 'golf ball', 'parachute']

# Select 25 random indices from the training set
random_indices = np.random.choice(len(train_images), size=25, replace=False)

plt.figure(figsize=(10,10))
for i, idx in enumerate(random_indices):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[idx])
    plt.xlabel(class_names[train_labels[idx]])
plt.show()

### Model Configuration

In [ ]:
# CNN Layers
model = models.Sequential()

# Adding a convolutional layer with 32 filters of size 3x3
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(64,64, 3)))
model.add(layers.MaxPooling2D((2, 2)))

# Adding a convolutional layer with 64 filters of size 3x3
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))

# Adding a convolutional layer with 64 filters of size 3x3
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))

# Dense Layers
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

# Printing model summary
model.summary()

### Model Compilation and Training

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

### Feature Map Rendering

#### Utility Functions

In [12]:
def save_feature_maps(feature_maps, output_dir, square=8):
    """
    Save feature maps from a convolutional layer as jpeg images
    
    Args:
        feature_maps: numpy array of shape (1, height, width, n_filters)
        output_dir: directory to save the feature map images
        square: number of feature maps to save
    """
    
    os.makedirs(output_dir, exist_ok=True)
    
    for i in range(min(square * square, feature_maps.shape[-1])):
        # Normalize to 0-255 range
        fm = feature_maps[0, :, :, i]
        fm = ((fm - fm.min()) * 255 / (fm.max() - fm.min())).astype(np.uint8)
        
        # Convert to PIL Image and save
        img = Image.fromarray(fm)
        img.save(os.path.join(output_dir, f'feature_map_{i}.jpg'))

def plot_feature_maps(feature_maps, square=8, figsize=(20, 20)):
    """
    Plot feature maps from a convolutional layer
    
    Args:
        feature_maps: numpy array of shape (1, height, width, n_filters)
        square: number of feature maps per row/column
        figsize: size of the figure
    """
    plt.figure(figsize=figsize)
    
    for i in range(min(square * square, feature_maps.shape[-1])):
        ax = plt.subplot(square, square, i + 1)
        ax.set_xticks([])
        ax.set_yticks([])
        plt.imshow(feature_maps[0, :, :, i], cmap='gray')
        plt.title(f'Filter {i}')
    
    plt.tight_layout()
    plt.show()

#### Rendering Feature Maps

In [ ]:
img_path = "/Users/ericcui/repos/imagenette/imagenette2-320/train/n02102040/ILSVRC2012_val_00000665.JPEG"

# Use the function
model_activations = Model(inputs=model.inputs, outputs=model.layers[0].output)
img = load_img(img_path, target_size=(64, 64))
img = img_to_array(img)
img = img.reshape(1, 64, 64, 3)
feature_maps = model_activations.predict(img)
plot_feature_maps(feature_maps)
save_feature_maps(feature_maps, "feature_maps")

### Saving Model

In [ ]:
# Model path with timestamp
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
model_path = f"cnn_model_{timestamp}.h5"

# Save the model
model.save(model_path)